# Création des 3 modèles

### Bibliothèques

In [ ]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

### Constantes

In [ ]:
largeur_image = 600
hauter_image = 600
filenames = ['blocked','free','stopsign','vitesses','directions','pieton']
vitesses = ['speed1sign','speed2sign','speed3sign']
directions = ['moveright','moveleft']
filenames.sort()
vitesses.sort()
directions.sort()

### Création des bases de données
Pour pouvoir créer les différents modèles, il faut mettre dans un dossier:
- dataset_direction les images montrant les panneaux de directions (après rognage)
- dataset_vitesse les images montrant les panneaux de vitesse (après rognage)  

Il est important de garder la même classification dans ces dossiers: dossiers moveleft et moveright dans dataset_direction et dossiers speed1sign, speed2sign et speed3sign dans dataset_vitesse

<i> Si besoin de dézipper : </i>

In [ ]:
!unzip -q dataset.zip
!unzip -q dataset_vitesse.zip
!unzip -q dataset_direction.zip

In [ ]:
dataset = datasets.ImageFolder(
    'dataset',
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1), #ColorJitter(brightness=0, contrast=0, saturation=0, hue=0) hue = teinte
        transforms.Resize((largeur_image, hauter_image)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

dataset_direction = datasets.ImageFolder(
    'dataset_direction',
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((largeur_image, hauter_image)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

dataset_vitesse = datasets.ImageFolder(
    'dataset_vitesse',
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((largeur_image, hauter_image)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

### Séparation des données

La moitié des données sont utilisées pour créer le modèle et l'autre moité est utilisée pour tester le modèle et ajuster les paramètres

In [ ]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - len(dataset)//2, len(dataset)//2])
train_dataset_direction, test_dataset_direction = torch.utils.data.random_split(dataset_direction, [len(dataset_direction) - len(dataset_direction)//2, len(dataset_direction)//2])
train_dataset_vitesse, test_dataset_vitesse = torch.utils.data.random_split(dataset_vitesse, [len(dataset_vitesse) - len(dataset_vitesse)//2, len(dataset_vitesse)//2])

### Création de lots pour prendre les données par paquets

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=True, num_workers=0)

train_loader_direction = torch.utils.data.DataLoader(train_dataset_direction, batch_size=8, shuffle=True, num_workers=0)
test_loader_direction = torch.utils.data.DataLoader(test_dataset_direction, batch_size=8, shuffle=True, num_workers=0)

train_loader_vitesse = torch.utils.data.DataLoader(train_dataset_vitesse, batch_size=8, shuffle=True, num_workers=0)
test_loader_vitesse = torch.utils.data.DataLoader(test_dataset_vitesse, batch_size=8, shuffle=True, num_workers=0)

### Définition du modèle : réseaux de neurones

In [ ]:
model = models.alexnet(pretrained=True)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, len(filenames)) #le dernier paramètres correspond au nombre de sortie du réseau de neurones, donc ici au nombre de dossiers
device = torch.device('cuda') #cpu pour l'executer sur son ordinateur perso et cuda pour l'executer sur le jetbot
model = model.to(device)

In [ ]:
model_direction = models.alexnet(pretrained=True)
model_direction.classifier[6] = torch.nn.Linear(model_direction.classifier[6].in_features, len(directions))#len(directions) correspond au nbrs de panneaux direction
device = torch.device('cuda') 
model_direction = model_direction.to(device)

In [ ]:
model_vitesse = models.alexnet(pretrained=True)
model_vitesse.classifier[6] = torch.nn.Linear(model_vitesse.classifier[6].in_features, len(vitesses))#len(vitesses) correspond au nbrs de panneaux vitesses
device = torch.device('cuda')
model_vitesse = model_vitesse.to(device)

### Entrainement des 3 réseaux de neurones

Les trois blocs suivant permettent de créer les modèles dont nous nous servirons pour décider de la situation dans laquelle nous nous trouvons.  
- Le permier correspond à la première décision (panneaux, piéton ...)  
- Le second au cas où l'on a des panneaux de directions  
- Le dernier au cas où l'on a des panneaux de vitesses  

Attention !! Ces codes prennent du temps à être exécutées. Il peut être intéressant de réaliser ces calculs en local sur son ordinateur puis d'importer le fichier obtenu. D'autant plus qu'avec un grand nombre de photos, la mémoire du jetbot n'est pas toujours suffisante.

In [ ]:
NUM_EPOCHS = 30 #peut être augmenter afin de tester davantage de paramètres pour le modèle mais cela implique une augmentation significative du temps d'execution
BEST_MODEL_PATH = 'best_model.pth'
best_accuracy = 0.0

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(NUM_EPOCHS):
    
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
    
    test_error_count = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
    
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))
    print('%d: %f' % (epoch, test_accuracy))
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        #Pour faire le calcul sur un ordinateur il faut utiliser la commande suivante à la place de la précédente :
        #torch.save(model.state_dict(), BEST_MODEL_PATH, _use_new_zipfile_serialization=False)
        #car sinon, vous ne pourrez pas relire le modèle avec le robot
        best_accuracy = test_accuracy

In [ ]:
NUM_EPOCHS = 30
BEST_MODEL_PATH = 'best_model_direction.pth'
best_accuracy = 0.0

optimizer = optim.SGD(model_direction.parameters(), lr=0.001, momentum=0.9)

for epoch in range(NUM_EPOCHS):
    
    for images, labels in iter(train_loader_direction):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model_direction(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
    
    test_error_count = 0.0
    for images, labels in iter(test_loader_direction):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model_direction(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
    
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset_direction))
    print('%d: %f' % (epoch, test_accuracy))
    if test_accuracy > best_accuracy:
        torch.save(model_direction.state_dict(), BEST_MODEL_PATH)
        #Pour faire le calcul sur un ordinateur il faut utiliser la commande suivante à la place de la précédente :
        #torch.save(model_direction.state_dict(), BEST_MODEL_PATH, _use_new_zipfile_serialization=False)
        #car sinon, vous ne pourrez pas relire le modèle avec le robot
        best_accuracy = test_accuracy

In [ ]:
NUM_EPOCHS = 30
BEST_MODEL_PATH = 'best_model_vitesse.pth'
best_accuracy = 0.0

optimizer = optim.SGD(model_vitesse.parameters(), lr=0.001, momentum=0.9)

for epoch in range(NUM_EPOCHS):
    
    for images, labels in iter(train_loader_vitesse):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model_vitesse(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
    
    test_error_count = 0.0
    for images, labels in iter(test_loader_vitesse):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model_vitesse(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
    
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset_vitesse))
    print('%d: %f' % (epoch, test_accuracy))
    if test_accuracy > best_accuracy:
        #Pour faire le calcul sur un ordinateur il faut utiliser la commande suivante à la place de la précédente :
        #torch.save(model_vitesse.state_dict(), BEST_MODEL_PATH, _use_new_zipfile_serialization=False)
        #car sinon, vous ne pourrez pas relire le modèle avec le robot
        best_accuracy = test_accuracy